In [ ]:
import os
import pandas as pd
import plotly.graph_objects as go
from ipywidgets import widgets
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 98%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [ ]:
# Case selection
CRV_DIR = "/home/marinjl/work/PtFige-Marseille/02092020.FALCON_RESULTS/gens/crv"
PREFIX = "gen_"

In [ ]:
# Read the metrics
metrics_dir = CRV_DIR + "/../metrics"
ast_metrics = metrics_dir + "/curve_metrics_ast.csv"
dwo_metrics = metrics_dir + "/curve_metrics_dwo.csv"
diff_metrics = metrics_dir + "/curve_diffmetrics.csv"
if not (os.path.isdir(metrics_dir) and
        os.path.isfile(ast_metrics) and
        os.path.isfile(dwo_metrics) and
        os.path.isfile(diff_metrics)        
       ):
    raise ValueError("Input datafiles (metrics) not found for %s" % crv_dir)

all_ast = pd.read_csv(ast_metrics, sep=";", index_col=False, compression="infer")
all_dwo = pd.read_csv(dwo_metrics, sep=";", index_col=False, compression="infer")
delta = pd.read_csv(diff_metrics, sep=";", index_col=False, compression="infer")
delta

In [ ]:
# Auxiliary function for reading curve data of each individual case
TFIN_TIME_OFFSET = 4000  # Dynawo's time offset w.r.t. Astre
def get_curve_dfs(crv_dir, prefix, contg_case):
    ast_case = crv_dir + "/" + prefix + contg_case + "-AstreCurves.csv.xz"
    dwo_case = crv_dir + "/" + prefix + contg_case + "-DynawoCurves.csv.xz"
    df_ast = pd.read_csv(ast_case, sep=";", index_col=False, compression="infer")
    df_dwo = pd.read_csv(dwo_case, sep=";", index_col=False, compression="infer")
    df_dwo = df_dwo.iloc[:, :-1]  # because of extra ";" at end-of-lines
    df_dwo["time"] = df_dwo.time - TFIN_TIME_OFFSET
    return df_ast, df_dwo

In [ ]:
# Combo Boxes
mask_n=['NETWORK' in x for x in all_dwo.vars]
df = delta[mask_n]

mask = widgets.Dropdown(
    options=list(['NETWORK','U_IMPIN','levelK','PGen','QGen']),
    value='NETWORK',
    description='Variable Group: ',
)

var = widgets.Dropdown(
    options=list(['dSS','dPP','TT','period','damp']),
    value='dSS',
    description='Metric: ',
)

contg_cases = list(delta['dev'].unique())
contg_case0 = contg_cases[0]

dev = widgets.Dropdown(
    options=contg_cases,
    value=contg_case0,
    description='Contg Case: '
)

# Load the curve data for the first case
df_ast, df_dwo = get_curve_dfs(CRV_DIR, PREFIX, contg_case0)

vars_ast = df_ast.columns[1:]
vars_dwo = df_dwo.columns[1:]

var0 = vars_ast[0]

var2 = widgets.Dropdown(
    options=vars_ast,
    value=var0,
    description='Variable: '
)

In [ ]:
# Traces
trace = go.Scatter(x=df['dSS_ast'],
                   y=df['dSS_dwo'],
                   mode='markers',
                   marker_color=df['TT_ast'],
                   marker_size=(df.dPP_ast - min(df.dPP_ast)) / (max(df.dPP_ast) - min(df.dPP_ast))*100,
                   text=df['dev'] + '<br>' + df['vars'],
                   xaxis="x1",
                   yaxis="y1"
                   )

trace1 = go.Scatter(x=df_ast['time'],
                   y=df_ast[var0],
                   mode='lines+markers',
                   marker_color='black',
                   xaxis="x2",
                   yaxis="y2"
                   )

trace2 = go.Scatter(x=df_dwo['time'],
                   y=df_dwo[var0] - df_dwo[var0][0] + df_ast[var0][1],
                   mode='lines',
                   marker_color='red',
                   xaxis="x2",
                   yaxis="y2"
                   )

In [ ]:
# Plot layout
layout = go.Layout(
    title=dict(text='Astre vs Dynawo'),
    yaxis=dict(
        domain=[0.3, 1],
        #anchor = 'x1'
    ),
    yaxis2=dict(domain=[0, 0.2]),
    xaxis2=dict(anchor = 'y2'),
    height=1000,
    width=800
)


g = go.FigureWidget(data=[trace,trace1,trace2],
                    layout=layout)
scatter = g.data[0]

In [ ]:
# Callbacks
def response(change):
    mask_=[mask.value in x for x in all_dwo.vars]
    df = delta[mask_]
    with g.batch_update():
            g.data[0].x = df[var.value + '_ast']
            g.data[0].y = df[var.value + '_dwo']
            g.data[0].marker.color = df.TT_ast
            g.data[0].marker.size = (df.dPP_ast - min(df.dPP_ast)) / (max(df.dPP_ast) - min(df.dPP_ast))*50
            g.data[0].text = df['dev'] + '<br>' + df['vars']
            g.layout.xaxis.title = var.value + 'Astre'
            g.layout.yaxis.title = var.value + 'Dynawo'


def response2(change):
    df_ast, df_dwo =  get_curve_dfs(CRV_DIR, PREFIX, dev.value)
    vars_ast = df_ast.columns[1:]
    var2.options=vars_ast
    var0 = vars_ast[0]
    var2.value = var0
    with g.batch_update():
            g.data[1].x = df_ast['time']
            g.data[1].y = df_ast[var0]
            g.data[2].x = df_dwo['time']
            g.data[2].y = df_dwo[var0] - df_dwo[var0][0] + df_ast[var0][1]


def response3(change):
    df_ast, df_dwo = get_curve_dfs(CRV_DIR, PREFIX, dev.value) 
    with g.batch_update():
            g.data[1].y = df_ast[var2.value]
            g.data[2].y = df_dwo[var2.value] - df_dwo[var2.value][0] + df_ast[var2.value][1]


def update_serie(trace, points, selector):
    t = list(scatter.text)
    for i in points.point_inds:
        #print(scatter.text[i])
        with g.batch_update():
            dev0 = scatter.text[i].split('<')[0]
            dev1 = scatter.text[i].split('>')[1]
            dev.value = dev0
            var2.value = dev1


var.observe(response, names="value")
mask.observe(response, names="value")            
dev.observe(response2, names="value")
var2.observe(response3, names="value")
scatter.on_click(update_serie)

In [ ]:
# Plot
container = widgets.HBox([mask, var, dev ,var2])
widgets.VBox([container,g])